# Colab 사용자를 위한 안내

해당 노트북은 **로컬** 환경에서 최적화 되어 있습니다. 로컬 환경에서 진행하시는 분들은 바로 학습을 진행하시면 됩니다.

Colab 을 사용하시는 분들은 처음에 아래 주석을 해제하시고 한번 만 실행시켜주세요!

* 주석을 해제하는 방법: 해당 영역을 선택하고, `Ctrl + /` 를 누르면 해당 영역의 주석에 해제됩니다.

In [2]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Mounted at /content/gdrive


Colab 을 사용하시는 분들은 아래 주석을 해제하시고 `folder` 변수 명에 프로젝트 디렉토리를 저장한 위치를 작성해주세요! 예를 들어, `03_rnn_pt` 의 위치가 "내 드라이브 > colab_notebook > pytorch" 폴더 안에 있는 경우, "colab_notebook/tensorflow" 를 작성하시면 됩니다.

```python
folder = "colab_notebook/pytorch"
```

In [3]:
import os
from pathlib import Path

# folder 변수에 구글드라이브에 프로젝트를 저장한 디렉토리를 입력하세요!
folder = "pytorch_colab"
project_dir = "week7"

base_path = Path("/content/gdrive/My Drive/")
project_path = base_path / folder / project_dir
os.chdir(project_path)
for x in list(project_path.glob("*")):
    if x.is_dir():
        dir_name = str(x.relative_to(project_path))
        os.rename(dir_name, dir_name.split(" ", 1)[0])
print(f"현재 디렉토리 위치: {os.getcwd()}")

현재 디렉토리 위치: /content/gdrive/My Drive/pytorch_colab/week7


PyTorch 는 `1.1.0` 이상 버전을 기준으로 합니다. Colab 사용시, 첫번째 코드를 실행해보시고 만약에 버전이 다르다면 두 번째 주석을 해제하고 실행해주세요.

In [4]:
import torch 
print('pytorch version: {}'.format(torch.__version__))

pytorch version: 1.10.0+cu111


In [ ]:
# !pip install torch==1.1.0 torchvision==0.3.0

# Recurrent Neural Network: Neural Weather Forecaster

<img src="http://drive.google.com/uc?export=view&id=139sr_X8Hi8O8s43syKDxW7UCF33X_mjh" width="600px" height="800px" />

* 이미지 출처: 네이버
<br>
많은 사람들이 아침에 집을 나서기 전에 오늘 기온이 어떤지 혹은 비가 오는지 알기 위해 일기예보를 확인합니다. 그런데 혹시 일기예보가 어떻게 이루어지는지 생각해 보신적이 있으신가요? 아직 오지도 않은 미래의 날씨를 어떻게 예측할 수 있을까요? 아마 여러분 대부분은 기상 예측과 관련된 전문적인 지식에 대해 잘 알지 못할 것입니다. 엄청난 계산능력을 갖춘 슈퍼컴퓨터가 복잡한 계산을 통해 예측을 한다는 정도는 들어보신 분들도 계실 수 있겠네요. 하지만 딥러닝을 활용할 수 있고, 지난 과거의 기후 데이터만 가지고 있으면 여러분의 PC에서도 훌륭한 일기예보 모델을 학습시킬 수 있습니다. 이번 프로젝트에서는 RNN을 직접 설계하여 24시간 후의 기온을 예측하는 문제를 해결할 것입니다.

이번 실습의 목표는 다음과 같습니다.
- RNN을 설계하여 지난 며칠 동안의 날씨 정보를 기반으로 24시간 이후의 기온을 예측한다.
- 다양한 속성의 시계열 정보를 활용하기 위해 적절한 전처리 과정을 적용한다. 
- 설계한 모델의 성능을 검증하기 위해 베이스라인 모델을 도입한다.

실습코드는 Python 3.6, Pytorch 1.1.0 버전을 기준으로 작성되었습니다.

이번 과정을 통해 얻는 최종 결과물은 아래 그림과 같습니다.

<img src="http://drive.google.com/uc?export=view&id=1UD4n1qLY2o3ayAQq-LOOiHd76qiCVAYk" width="600px" height="400px" />
<center>&lt;기온, 강수량 등 다양한 정보로 미래의 기온 예측&gt;</center></caption>

### 이제부터 본격적으로 프로젝트를 시작하겠습니다.

**"[TODO] 코드 구현"** 부분의 **"## 코드 시작 ##"** 부터 **"## 코드 종료 ##"** 구간에 필요한 코드를 작성해주세요. **나머지 작성구간이 명시 되지 않은 구간은 임의로 수정하지 마세요!**

**본문 중간중간에 Pytorch 함수들에 대해 [Pytorch API 문서](https://pytorch.org/docs/stable/) 링크를 걸어두었습니다. API 문서를 직접 확인하는 일에 익숙해지면 나중에 여러분이 처음부터 모델을 직접 구현해야 할 때 정말 큰 도움이 됩니다.**

<h1>목차<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Colab-사용자를-위한-안내" data-toc-modified-id="Colab-사용자를-위한-안내-1">Colab 사용자를 위한 안내</a></span></li><li><span><a href="#Recurrent-Neural-Network:-Neural-Weather-Forecaster" data-toc-modified-id="Recurrent-Neural-Network:-Neural-Weather-Forecaster-2">Recurrent Neural Network: Neural Weather Forecaster</a></span><ul class="toc-item"><li><span><a href="#1.-Package-load" data-toc-modified-id="1.-Package-load-2.1">1. Package load</a></span></li><li><span><a href="#2.-하이퍼파라미터-세팅" data-toc-modified-id="2.-하이퍼파라미터-세팅-2.2">2. 하이퍼파라미터 세팅</a></span></li><li><span><a href="#3.-데이터-전처리-함수-정의" data-toc-modified-id="3.-데이터-전처리-함수-정의-2.3">3. 데이터 전처리 함수 정의</a></span></li><li><span><a href="#4.-Dataset-정의-및-DataLoader-할당" data-toc-modified-id="4.-Dataset-정의-및-DataLoader-할당-2.4">4. Dataset 정의 및 DataLoader 할당</a></span></li><li><span><a href="#5.-데이터-샘플-시각화" data-toc-modified-id="5.-데이터-샘플-시각화-2.5">5. 데이터 샘플 시각화</a></span></li><li><span><a href="#6.-베이스라인-성능-측정" data-toc-modified-id="6.-베이스라인-성능-측정-2.6">6. 베이스라인 성능 측정</a></span></li><li><span><a href="#7.-네트워크-설계" data-toc-modified-id="7.-네트워크-설계-2.7">7. 네트워크 설계</a></span></li><li><span><a href="#8.-train,-validation,-test-함수-정의" data-toc-modified-id="8.-train,-validation,-test-함수-정의-2.8">8. train, validation, test 함수 정의</a></span></li><li><span><a href="#9.-모델-저장-함수-정의" data-toc-modified-id="9.-모델-저장-함수-정의-2.9">9. 모델 저장 함수 정의</a></span></li><li><span><a href="#10.-모델-생성-및-Loss-function,-Optimizer-정의" data-toc-modified-id="10.-모델-생성-및-Loss-function,-Optimizer-정의-2.10">10. 모델 생성 및 Loss function, Optimizer 정의</a></span></li><li><span><a href="#11.-Training" data-toc-modified-id="11.-Training-2.11">11. Training</a></span></li><li><span><a href="#12.-저장된-모델-불러오기-및-테스트" data-toc-modified-id="12.-저장된-모델-불러오기-및-테스트-2.12">12. 저장된 모델 불러오기 및 테스트</a></span></li><li><span><a href="#13.-Summary" data-toc-modified-id="13.-Summary-2.13">13. Summary</a></span></li></ul></li><li><span><a href="#Self-Review" data-toc-modified-id="Self-Review-3">Self-Review</a></span></li></ul></div>

## 1. Package load
필요한 패키지들을 로드합니다.

In [5]:
import os
import glob
import csv
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import check_util.checker as checker
%matplotlib inline

print('pytorch version: {}'.format(torch.__version__))
print('GPU 사용 가능 여부: {}'.format(torch.cuda.is_available()))
device = "cuda" if torch.cuda.is_available() else "cpu"   # GPU 사용 가능 여부에 따라 device 정보 저장

pytorch version: 1.10.0+cu111
GPU 사용 가능 여부: False


## 2. 하이퍼파라미터 세팅
학습에 필요한 하이퍼파리미터의 값을 초기화해줍니다.

미니배치의 크기(`batch_size`), 학습 할 세대(epoch) 수(`num_epochs`), 학습률(`learning_rate`) 등의 값들을 다음과 같이 정했습니다.

In [6]:
batch_size = 100
num_epochs = 30
learning_rate = 0.00003

## 3. 데이터 전처리 함수 정의

우리는 이번 실습에서 지난 10년간의(2009년~2018년) 서울시 기후 데이터를 활용해 기온을 예측하는 모델을 학습시킬 것입니다. 데이터셋은 [기상자료개방포털](https://data.kma.go.kr/)에서 받은 자료입니다. 이번 실습에서 사용하는 데이터 이외에도 기상자료개방포털에서 기상과 관련된 다양한 자료들을 내려받으실 수 있습니다. 

`./data/climate_seoul` 경로의 디렉토리를 보시면, `test` / `train` / `val` 디렉토리에 csv파일이 각각 1개 / 8개 / 1개 담겨있음을 확인하실 수 있습니다. 각 csv파일은 1년간의 서울시 기후 데이터를 담고 있으며, 1시간 간격으로 기록된 정보입니다. (사실 아주 가끔씩 30분 간격으로 기록한 구간도 있기도 하지만, 이후 본문에서는 편의상 모두 1시간 간격으로 기록된 것으로 간주하겠습니다.) 매 시간마다 기록되는 정보는 기온, 강수량, 풍속 등을 포함한 총 25가지 속성으로 이루어져 있습니다. 이 중에서 우리는 기온, 강수량, 풍속, 습도, 증기압을 포함한 총 9가지의 속성만을 사용하여 기온 예측 모델을 학습시켜 보겠습니다.

그렇다면 왜 하필 이 9가지의 속성을 선택한 것일까요? 이렇게 9가지의 속성을 선택한 배경에는 어떠한 전문적인 지식도 고려되지 않은 것입니다. 25가지의 속성을 모두 사용해볼 수도 있겠죠. 어떤 속성들을 활용할지는 설계자의 몫입니다. 그런데 여러분이 기상과 관련된 전문적인 지식을 갖고 있지 않는 이상 이중에서 어떤 속성이 기온 예측에 가장 중요한지, 또는 어떤 속성이 가장 불필요한 속성인지 알지 못할 것입니다. 하지만 고맙게도 딥러닝은 이러한 속성 선택 문제에 덜 예민한 학습 방식입니다. 더 정확하게 말하면, 다소 불필요한 정보가 입력으로 주어진다고 해서 극단적으로 학습이 이루어지지 않는 일은 일어나지 않을 가능성이 큽니다. 학습과정에서 인공신경망이 필요한 특징(feature)을 알아서 추출하기 때문입니다. 그러니까 우리는 어떤 속성을 활용할지를 너무 심각하게 고민하지 않아도 되는 것입니다. 다만 이번 실습에서는 매번 빠짐없이 잘 기록된 속성들을 위주로 9가지를 선택한 것 뿐입니다.

아래에 정의한 전처리 `preprocess` 함수는 csv파일들을 읽어 9가지 속성 정보만을 NumPy 배열에 저장해 반환하는 역할을 합니다. 이 메소드는 이후에 구현할 Dataset class에서 활용할 것입니다.

In [7]:
def preprocess(all_files):
    data_0 = [] # 기온
    data_1 = [] # 강수량
    data_2 = [] # 풍속
    data_3 = [] # 습도
    data_4 = [] # 증기압
    data_5 = [] # 이슬점 온도
    data_6 = [] # 현지 기압
    data_7 = [] # 해면 기압
    data_8 = [] # 지면 온도
    for f in all_files:
        with open(f, encoding='euc-kr') as c:
            csv_reader = csv.reader(c, delimiter=',')
            header = True
            for col in csv_reader:
                if header:
                    header = False
                    continue
                data_0.append(float(col[2])) if col[2] != '' else data_0.append(0.0)
                data_1.append(float(col[3])) if col[3] != '' else data_1.append(0.0)
                data_2.append(float(col[4])) if col[4] != '' else data_2.append(0.0)
                data_3.append(float(col[6])) if col[6] != '' else data_3.append(0.0)
                data_4.append(float(col[7])) if col[7] != '' else data_4.append(0.0)
                data_5.append(float(col[8])) if col[8] != '' else data_5.append(0.0)
                data_6.append(float(col[9])) if col[9] != '' else data_6.append(0.0)
                data_7.append(float(col[10])) if col[10] != '' else data_7.append(0.0)
                data_8.append(float(col[22])) if col[22] != '' else data_8.append(0.0)

    data = np.zeros((len(data_0), 9))
    for i, d in enumerate(data):
        data[i, 0] = data_0[i]
        data[i, 1] = data_1[i]
        data[i, 2] = data_2[i]
        data[i, 3] = data_3[i]
        data[i, 4] = data_4[i]
        data[i, 5] = data_5[i]
        data[i, 6] = data_6[i]
        data[i, 7] = data_7[i]
        data[i, 8] = data_8[i]
    return data

`preprocess` 를 해주고 나면, 데이터가 다음 그림과 같이 매 시간마다 총 9가지의 속성만 저장됩니다.

<img src="http://drive.google.com/uc?export=view&id=1iA_SHcfJ9XnZ2w_S66GS_XpvRH7WNXGf" width="600px" height="400px" />


## 4. Dataset 정의 및 DataLoader 할당

이제 우리가 사용할 데이터셋에 대해 정의할 차례입니다. PyTorch 의 Dataset과 DataLoader에 대해 잘 기억나지 않는다면 ['Lab-04-2'](https://www.youtube.com/watch?v=B3VG-TeO9Lk&list=PLQ28Nx3M4JrhkqBVIXg-i5_CVVoS1UzAv&index=8&t=0s) 강의를 참고하시기 바랍니다.


생성자(`__init__`)는 미리 구현을 해두었습니다. 여기서 사용되는 각 변수의 의미는 다음과 같습니다.
- `seq_len`: 우리가 이후에 설계할 RNN의 입력으로 줄 데이터 시퀀스의 길이, 즉 총 타임스텝의 길이를 뜻합니다. 다시 말하면, 현재로부터 24시간 뒤의 기온을 예측하기 위해 얼마만큼의 과거 정보를 참고할지 결정하는 것입니다. 기본값을 480로 해두었는데, 이는 480개의 과거 데이터를 살펴보겠다는 것이고, 데이터 1개는 1시간마다 기록되기 때문에 결과적으로 지난 20일간의 데이터를 기반으로 24시간 후의 기온을 예측하겠다는 의미입니다. 
- `target_delay`: 우리가 예측할 시점이 입력 시퀀스의 마지막 타임스텝으로 얼만큼 이후인지 결정하는 것입니다. 24로 기본값을 해두었고, 이는 24시간 이후의 기온이 우리가 예측할 대상임을 의미하는 것입니다. 예를 들어 1월 1일 00:00 부터 1월 21 일 23:00 까지 480 개의 데이터를 이용한다면 예측 데이터는 1월 21일 23:00 로부터 24 시간 후인 1월 22일 23:00 시점의 기온입니다. 
- `stride`: 데이터를 모델에게 입력으로 주기 위해 우리는 `seq_len` 길이 만큼의 정보를 우리가 가진 전체 데이터에서 임의로 선택해야 합니다. 임의로 선택된 그 시작 지점을 시작 인덱스라고 부른다면, `stride`는 그 시작 인덱스 후보들 간의 간격을 의미합니다. `stride=1`이라면 모든 시점의 데이터가 시작 인덱스가 될 수 있는 것이고, `stride=2`이면 아래의 그림처럼 시작 인덱스 후보가 하나씩 건너띄어 존재하므로, 전체 데이터 포인트에서 절반만이 시작 인덱스가 될 수 있습니다. `stride`가 작을수록 모델의 서로 다른 입력간에 정보가 중복되는 정도가 크겠죠. <br> 예를 들어, `seq_len=5` 인 상황에서 `stride=1` 이면 입력 데이터는 $\mathbf{x}^{(1)} = [x_{1}, x_{2}, \cdots, x_{5}]$, $\mathbf{x}^{(2)} = [x_{2}, x_{3}, \cdots, x_{6}]$, $\mathbf{x}^{(3)} = [x_{3}, x_{4}, \cdots, x_{7}]$ 형태가 되고,`stride=2` 면 입력 데이터는 $\mathbf{x}^{(1)} = [x_{1}, x_{2}, \cdots, x_{5}]$, $\mathbf{x}^{(2)} = [x_{3}, x_{4}, \cdots, x_{7}]$, $\mathbf{x}^{(3)} = [x_{5}, x_{6}, \cdots, x_{9}]$가 됩니다. <br> 어떤 값으로 정할지는 역시 설계자인 우리의 몫입니다. 이번 예제에서는 기본값을 5로 정하겠습니다. 

<img src="http://drive.google.com/uc?export=view&id=1DJ6lybrsRD8D8TtHBvbo-DU7zi-U4JMm"  width="800px" height="300px" />

- `all_files`: 정의할 Dataset에 사용할 모든 csv파일의 경로를 담고 있습니다. `data_dir`은 데이터셋의 디렉토리 경로를 의미하고 `mode`는 정의하고자 하는 Dataset에 따라 'train' 또는 'val' 또는 'test'으로 구분될 것입니다. 
- `self.data`: 위에서 정의한 데이터 전처리 메소드인 `preprocess`에서 데이터를 전처리한 결과가 저장됩니다. `self.data`의 shape은 (데이터의 총 길이, 9)가 됩니다.
- `normalize`: 입력으로 사용하는 데이터의 각 속성은 저마다 값의 범위가 다릅니다. 예를들어 기온은 보통 -15에서 35사이의 값을 갖지만, 강수량과 풍속은 음수 값이 존재하지 않고, 기압의 경우에는 1000 내외의 값이 일반적입니다. 이러한 경우 각각의 속성들을 저마다의 평균(mean)과 표준편차(std)를 통해 값을 정규화해주는 것이 바람직합니다. 


### <font color='red'>[TODO] 코드 구현</font>

이제 다음을 읽고 코드를 완성해보세요.

1. `self.mean`, `self.std`: normalize 실행시 검증, 테스트 단계에서는 훈련 데이터로부터 구한 평균과 분산 값을 사용해야합니다. 따라서 훈련 데이터 생성시(`mode == 'train'` 일때), `train` 폴더어 있는 데이터로부터 가져온 `self.data` 를 이용해 평균과 분산을 구하고 `self.mean`, `self.std`에 저장합니다. 그 후 검증, 테스트 데이터셋을 만들때, 이를 전달하여 normalize를 실행합니다. `self.data` 는 2 차원 NumPy 행렬이라는 것을 명심하시고, `mode == 'train'` 일때는 평균과 분산을 구하고, 그렇지 않으면 전달 받은 `mean` 과 `std` 를 `self.mean`, `self.std` 에 저장하세요. 함수를 사용할 경우 `axis`를 주의해 주세요.
2. `__len__`은 미리 구현을 해두었고, 이제 여러분이 직접 `__getitem__`을 구현해야 합니다. 모델의 입력으로 줄 데이터 시퀀스를 `sequence` 변수에 저장하고, 그 `sequence`의 가장 마지막 타임스텝으로부터 24시간 후의 기온을 `target` 변수에 저장하여 코드를 완성해보세요.
    * 시작인덱스 후보인 `index` 변수는 `__len__`에서 정의한 데이터의 총 길이를 값의 범위로 합니다. 예를 들어 현재 가진 `self.data`의 데이터 개수가 `1000`이고 `seq_len=480`, `stride=3`,`target_delay=24`라면, `__len__`는 1000에서 마지막 504개의 데이터를 제외하고 3을 나눈 몫을 반환하며, `index`는 0~164 사이의 값을 가지게 됩니다. 
    * 2-1 : `sequence`를 정의하기 위해서는 전체 데이터, 즉 `self.data`에서 시작 인덱스부터 `seq_len` 길이만큼을 인덱싱을 해서 가져와야 합니다. 이때 시작 인덱스로 모델에게 전달하는 `index` 변수를 바로 사용해서는 안됩니다. 앞서 언급한 것처럼 이번 문제에서 정의한 `index`는 시작 인덱스 후보의 총 갯수 사이의 값입니다. 따라서 `index`에 `self.stride`를 곱해주고 다시 `index` 변수에 저장하세요.
    * 2-2 : `self.data` 에서 인덱싱을 통해 입력 데이터인 `sequence`를 만들어보세요. `sequence`의 shape은 (`seq_len`, 9)가 되어야 합니다. 2-1 에서 정의한 `index`, `self.seq_len` 변수가 필요합니다.
    * 2-4 : `self.data` 에서 인덱싱을 통해 타겟 데이터인 `target`을 만들어보세요. 2-1 에서 정의한 `index`, `self.seq_len`, `target_delay` 변수가 필요합니다. 파이썬에서 첫번째 인덱스는 숫자 0으로 시작한다는 것도 유의하세요!
    * 2-4 : 타겟 데이터인 `target`의 shape은 (1)이 되어야 합니다.첫번째 차원에 축을 추가하세요. 데이터에 축을 삽입해 차원을 증가시키고자 한다면 NumPy의 [expand_dims()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.expand_dims.html)를 활용할 수 있습니다.

**이제 모델에게 전달할 데이터 공급 코드를 작성해보세요! "<font color='45A07A'>## 코드 시작 ##</font>"과 "<font color='45A07A'>## 코드 종료 ##</font>" 사이의 `None` 부분을 채우시면 됩니다.**

In [ ]:
class Dataset(Dataset):
    def __init__(self, data_dir, mode, mean=None, std=None, seq_len=480, target_delay=24, stride=5, normalize=True):
        self.mode = mode
        self.seq_len = seq_len
        self.target_delay = target_delay
        self.stride = stride
        all_files = sorted(glob.glob(os.path.join(data_dir, mode, '*')))
        self.data = preprocess(all_files)
        if mode == 'train':
            assert (mean is None) and (std is None), \
                "평균과 분산은 train 폴더의 있는 데이터로 구하기 때문에 None 으로 설정합니다."
            ## 코드 시작 ##
            self.mean = None    # 위의 설명 1. 을 참고하여 None을 채우세요.
            self.std = None     # 위의 설명 1. 을 참고하여 None을 채우세요.
            ## 코드 종료 ##
        else:
            assert (mean is not None) and (std is not None), \
                "평균과 분산은 `train_data`변수에 내장한 self.mean 과 self.std 를 사용합니다."
            ## 코드 시작 ##
            self.mean = None    # 위의 설명 1. 을 참고하여 None을 채우세요.
            self.std = None     # 위의 설명 1. 을 참고하여 None을 채우세요.
            ## 코드 종료 ##
        
        if normalize:
            self.data = (self.data - self.mean) / self.std
    
    def __getitem__(self, index):
        ## 코드 시작 ##
        index = None            # 위의 설명 2-1 을 참고하여 None을 채우세요.
        sequence = None         # 위의 설명 2-2 를 참고하여 None을 채우세요.
        target = None           # 위의 설명 2-3 을 참고하여 None을 채우세요.
        target = None           # 위의 설명 2-4 를 참고하여 None을 채우세요.
        ## 코드 종료 ##
        return sequence, target
    
    def __len__(self):
        max_idx = len(self.data) - self.seq_len - self.target_delay
        num_of_idx = max_idx // self.stride
        return num_of_idx

이제 학습용, 검증용, 테스트용 Dataset과 DataLoader를 각각 할당합니다. 

In [ ]:
data_dir = './data/climate_seoul'
train_data = Dataset(data_dir, 'train', mean=None, std=None)
val_data = Dataset(data_dir, 'val', mean=train_data.mean, std=train_data.std)
test_data = Dataset(data_dir, 'test', mean=train_data.mean, std=train_data.std)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, drop_last=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, drop_last=True)

아래의 코드를 실행해 코드를 성공적으로 완성했는지 확인해보세요. 

별다른 문제가 없다면 이어서 진행하면 됩니다.

In [ ]:
checker.customized_dataset_check(train_data)

## 5. 데이터 샘플 시각화

`train_data`의 첫번째 `sequence`에서 기온 정보만을 그래프 형태로 시각화합니다. 

In [ ]:
temp = train_data[0][0]
temp = temp[:, 0]
plt.plot(range(len(temp)), temp)
plt.xlabel('time')
plt.ylabel('temperature\n(normalized)')
plt.show()

## 6. 베이스라인 성능 측정

이후에 우리가 학습시킨 모델의 성능의 좋고 나쁨을 판단할 기준(베이스라인)이 필요합니다. 적당한 수준의 기준 성능을 정해놓고 테스트 결과에서 우리의 모델이 그것보다 더 좋은 성능을 보이면 만족할만한 모델을 학습시킨 것으로 간주하면 되고, 반대로 그보다 성능이 좋지 않다면 모델을 더 개선시키는 방식으로 네트워크 구조를 변경해 나아가면 됩니다.

과거의 기후 정보를 활용하는 우리의 딥러닝 모델이 과연 의미있는 성능이라는 걸 보이기 위해서는 어떠한 예측 방식이 기준이 될 수 있을까요? 아마 기온 예측 전문가가 아닌 대부분의 여러분이 지금으로부터 24시간 후의 기온을 예측하라는 질문을 받았다고 생각해 봅시다. 아마 가장 무난하면서도 안정적으로 예측하는 방식은 지금의 기온과 같다고 답하는 것일 겁니다. 이러한 예측 방식은 과거의 기후 정보를 복잡하게 고려할 필요도 없이 간단하지만, 많은 경우에 실제로 꽤나 정확한 예측을 할 수 있는 방식입니다. 내일 낮 12시의 기온은 특이한 경우가 아니라면, 오늘 낮 12시의 기온과 거의 비슷할 것입니다. 

### <font color='red'>[TODO] 코드 구현</font>

아래의 `eval_baseline`은 베이스라인 모델의 평균 loss를 측정하는 함수입니다. 다음을 읽고 코드를 완성해보세요.

1. 우리의 베이스라인은 현재 기온을 24시간 후의 기온으로 예측하는 방식입니다. 현재 기후 정보는 `data_loader`에서 받은 `sequence`의 가장 마지막 타임스텝에 담겨있습니다. 입력으로 주는 `sequence`는 기온 뿐만아니라 총 9가지 속성이 포함된 것임에 유의하세요. 기온은 9가지 속성중 가장 첫번째에 위치하고 있음을 염두에 두고 `sequence`로 부터 현재 기온을 가져와 `pred` 변수에 저장하세요.
     * `pred`의 shape은 (batch_size, 1)이 되어야 합니다. 만약 여러분이 구한 `pred`의 shape이 (batch_size)라면, [torch.unsqueeze](https://pytorch.org/docs/stable/torch.html#torch.unsqueeze)를 활용하여 크기 1인 차원을 텐서에 삽입해보세요. 
2. `criterion` 은 차후에 넣을 손실함수(loss function)입니다. `criterion`을 함수의 첫번째 인자로 추정치인 `pred`, 두번째 인자로 실제 24시간 이후의 기온인 `target`을 이용하여 측정된 loss를 `loss` 변수에 저장하세요. (힌트: `criterion`는 함수 타입을 넣을 것임으로 다음과 같이 사용할 수 있습니다. <br> `함수(1번 인자, 2번인자, ...)`)

**eval_baseline 함수 코드를 작성해보세요! "<font color='45A07A'>## 코드 시작 ##</font>"과 "<font color='45A07A'>## 코드 종료 ##</font>" 사이의 <font color='075D37'>None</font> 부분을 채우시면 됩니다.**

In [ ]:
def eval_baseline(data_loader, criterion):
    total_loss = 0
    cnt = 0
    for step, (sequence, target) in enumerate(data_loader):
        ## 코드 시작 ##
        pred = None        # 위의 설명 1. 을 참고하여 None을 채우세요.
        loss = None        # 위의 설명 2. 를 참고하여 None을 채우세요.
        ## 코드 종료 ##
        total_loss += loss
        cnt += 1
    avrg_loss = total_loss / cnt
    print('Baseline Average Loss: {:.4f}'.format(avrg_loss))
    return avrg_loss.item()

베이스라인의 성능을 측정합니다. 다음과 같은 결과가 출력된다면 성공적으로 구현한 것입니다.

> Baseline Average Loss: 0.0952

In [ ]:
baseline_loss = eval_baseline(test_loader, nn.MSELoss())

평균 loss만 봐서는 베이스라인이 어느 정도로 예측을 잘하는지 감이 잘 오지 않습니다. 베이스라인 모델의 예측 기온과 실제 기온을 몇가지 살펴보면 다음과 같습니다.

In [ ]:
for i in range(15):
    data_idx = np.random.randint(len(test_data))
    pred = test_data[data_idx][0][-1, 0]
    pred = pred * std[0] + mean[0]  # 예측 기온을 normalization 이전 상태(섭씨 단위)로 되돌리는 작업
    target = test_data[data_idx][1][0] * std[0] + mean[0]  # 실제 기온을 normalization 이전 상태(섭씨 단위)로 되돌리는 작업
    print('예측 기온: {:.1f} / 실제 기온: {:.1f}'.format(pred, target))

## 7. 네트워크 설계

우리는 LSTM 구조를 통해 기온 예측을 모델을 학습시킬 것입니다. 설계할 네트워크의 대략적인 개요는 아래 그림과 같습니다.

<img src="http://drive.google.com/uc?export=view&id=1AjxdDivdX23DSdubayKhAAcXmWfRBSvb"  width="800px" height="400px" />

LSTM의 매 타임스텝의 입력은 매 시간마다 기록된 9가지 속성값이 들어가게 됩니다. 그리고 마지막 타입스텝의 출력을 마무리로 Fully Connected 레이어에 넣어 최종 예측 기온값을 출력하는 구조입니다.  

### <font color='red'>[TODO] 코드 구현</font>

다음을 읽고 코드를 완성해보세요. 

1. [nn.LSTM](https://pytorch.org/docs/stable/nn.html#torch.nn.LSTM)을 활용하여 생성자에 LSTM 레이어를 선언하고 이를 `self.lstm` 변수에 저장하세요. LSTM 레이어 선언에 필요한 파라미터인 `input_size`에 적절한 값을 넣어보세요. 또 다른 파라미터인 `hidden_size`, `num_layers`에는 각각 `self.hidden_size`, `self.num_layers`를 넣어주세요. 그리고 `batch_first=True`로 하시기 바랍니다. `batch_first=True`이면 LSTM 레이어의 입력의 shape이 (batch_size, 전체 시퀀스 길이, input_size)가 됩니다. 기본값은 `batch_first=False`인데, 이 경우 shape 는 (전체 시퀀스 길이, batch_size, input_size) 가 됩니다. 
    - `__init__`의 파라미터 `hidden_size`는 우리가 설계할 LSTM 레이어의 hidden state의 크기입니다. 이 크기를 얼마를 할지는 역시 설계자의 몫입니다. 이번 예제에서 기본값은 100으로 하겠습니다. 
    - `num_layers`는 LSTM 레이어의 총 레이어 수입니다. 기본값은 1로 하겠습니다.
2. LSTM의 마지막 타임스텝의 출력을 입력으로 받아 최종 기온을 예측하는 FC 레이어를 선언하고 이를 `self.fc` 변수에 저장하세요. 
3. `init_hidden` 함수를 구현할 차례입니다. `init_hidden`은 초기 hidden state, cell state를 만들어 반환하는 함수입니다. 초기 hidden state, cell state는 일반적으로 0으로 채운 값을 사용합니다. 0으로 채운 텐서를 생성하기 위해서는 [torch.zeros](https://pytorch.org/docs/stable/torch.html?highlight=zeros#torch.zeros)를 활용하시기 바랍니다. hidden state와 cell state의 shape은 (num_layers, batch_size, hidden_size)가 되어야 합니다. 0으로 채운 hidden state와 cell state를 선언하여 각각 `hidden` `cell` 변수에 저장하세요. 
4. `forward` 함수를 구현할 차례입니다. `forward`의 파라미터 `x`, `h`, `c`는 각각 LSTM의 입력, LSTM의 초기 hidden state, 초기 cell state를 의미합니다. PyTorch의 LSTM 레이어는 `x, (h, c)` 를 입력으로 주면 LSTM의 모든 타입스텝의 출력, (마지막 타임스텝의 hidden state, 마지막 타임스텝의 cell state) 를 반환합니다. (hidden state와 cell state을 입출력으로 주고 받을 때에는 둘을 괄호로 묶어 tuple 형태여야 함에 유의하시기 바랍니다.)
5. LSTM의 출력(`out` 변수)의 마지막 타임스텝의 출력을 FC 레이어에 입력으로 주어 얻은 결과를 `final_output` 변수에 저장하세요. 

**모델 코드를 작성해보세요! "<font color='45A07A'>## 코드 시작 ##</font>"과 "<font color='45A07A'>## 코드 종료 ##</font>" 사이의 <font color='075D37'>None</font> 부분을 채우시면 됩니다.**

In [ ]:
class SimpleLSTM(nn.Module):
    def __init__(self, hidden_size=100, num_layers=1):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        ## 코드 시작 ##
        self.lstm = None        # 위의 설명 1. 을 참고하여 None을 채우세요.
        self.fc = None          # 위의 설명 2. 를 참고하여 None을 채우세요.
        ## 코드 종료 ##
    
    def init_hidden(self, batch_size):
        # 코드 시작
        hidden = None           # 위의 설명 3. 을 참고하여 None을 채우세요.
        cell = None             # 위의 설명 3. 을 참고하여 None을 채우세요.
        # 코드 종료
        return hidden, cell
    
    def forward(self, x):
        # hidden, cell state init
        h, c = self.init_hidden(x.size(0))
        h, c = h.to(x.device), c.to(x.device)
        ## 코드 시작 ##
        out, (h, c) = None      # 위의 설명 4. 를 참고하여 None을 채우세요.
        final_output = None     # 위의 설명 5. 를 참고하여 None을 채우세요.
        ## 코드 종료 ##
        return final_output


아래의 코드를 실행해 코드를 성공적으로 완성했는지 확인해보세요. 

별다른 문제가 없다면 이어서 진행하면 됩니다.

> 디버깅 팁: 실제로 임의의 텐서를 생성해서 모델에 넣어보세요! 최종 출력의 크기 등 확인 할 수 있습니다.
> ```python
x = torch.randn(알맞는 형태)
out = model(x)
print(out.size())
> ```

In [ ]:
checker.model_check(SimpleLSTM(), batch_size)

## 8. train, validation, test 함수 정의

이번에는 훈련, 검증, 테스트를 진행하는 함수를 정의하겠습니다.

### <font color='red'>[TODO] 코드 구현: 훈련 함수</font>

훈련 함수입니다. 다음을 읽고 코드를 완성해 보세요.

먼저 `sequence` `target`의 데이터 형식을 `torch.float32`로 변환해주었습니다. 이렇게 해주는 이유는 현재 구현한 DataLoader에서는 `torch.float64` 형식으로 데이터를 반환하는데, 모델의 파라미터는 기본적으로 `torch.float32`로 이루어져 있기 때문입니다. 모델의 파라미터와 입력 텐서가 같은 데이터 형식을 갖도록 해주어야 합니다. Torch.Tensor의 다양한 데이터 형식에 대해 알아보려면 [여기](https://pytorch.org/docs/stable/tensor_attributes.html#torch.torch.dtype)를 참고하시기 바랍니다. 


`train` 함수에 여러 인자들이 보입니다. 

1. 모델(`model`)에 입력 시퀀스(`sequence`)를 전달하고 출력 결과를 `outputs`에 저장합니다.
2. `criterion` 은 손실함수를 담은 객체입니다. 예측 값인 `outputs`와 타겟 값인 `target`를 통해 손실값을 계산하고 그 결과를 `loss` 변수에 저장합니다.
3. `optimizer`은 옵티마이저 입니다. 이전에 계산한 기울기를 모두 clear하고, 오차 역전파(backpropagation)를 통해 기울기를 계산하고, 옵티마이저를 통해 파라미터를 업데이트합니다. 

일정한 에폭마다 다음에 구현할 `validation` 함수를 통해 검증을 수행합니다. 모델 검증을 수행했을 때, 만약 검증 과정의 평균 loss가 현재까지 가장 낮다면 가장 잘 훈련된 모델로 가정하고 그때까지 학습한 모델을 저장합니다. 저장은 추후에 구현할 `save_model` 함수가 수행합니다. 

**tarin 함수를 작성해보세요! "<font color='45A07A'>## 코드 시작 ##</font>"과 "<font color='45A07A'>## 코드 종료 ##</font>" 사이의 <font color='075D37'>None</font> 부분을 채우시면 됩니다.**

In [ ]:
def train(num_epochs, model, data_loader, criterion, optimizer, saved_dir, val_every, device):
    print('Start training..')
    best_loss = 9999999
    for epoch in range(num_epochs):
        for step, (sequence, target) in enumerate(data_loader):
            sequence = sequence.type(torch.float32)
            target = target.type(torch.float32)
            sequence, target = sequence.to(device), target.to(device)
            ## 코드 시작 ##
            outputs = None  # 위의 설명 1. 을 참고하여 None을 채우세요.
            loss = None     # 위의 설명 2. 를 참고하여 None을 채우세요.

            None            # Clear gradients: 위의 설명 3. 을 참고하여 None을 채우세요.
            None            # Gradients 계산: 위의 설명 3. 을 참고하여 None을 채우세요.
            None            # Parameters 업데이트: 위의 설명 3. 을 참고하여 None을 채우세요.
            ## 코드 종료 ##
            
            if (step + 1) % 25 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
                    epoch+1, num_epochs, step+1, len(train_loader), loss.item()))
                
        if (epoch + 1) % val_every == 0:
            avrg_loss = validation(epoch + 1, model, val_loader, criterion, device)
            if avrg_loss < best_loss:
                print('Best performance at epoch: {}'.format(epoch + 1))
                print('Save model in', saved_dir)
                best_loss = avrg_loss
                save_model(model, saved_dir)

### <font color='red'>[TODO] 코드 구현: 검증 함수</font>

검증 함수입니다. 다음을 읽고 코드를 완성해보세요. 

- 검증 과정에서는 파라미터 업데이트를 하지 않기 때문에 기울기를 계산할 필요는 없습니다. 하지만 검증 과정에서의 평균 loss를 계산하기 위해 loss는 계산해야 합니다. 
- `train` 함수와 마찬가지로 `model`에 입력 시퀀스(`sequence`)를 전달하고 출력 결과를 `outputs`에 저장하고, `criterion`을 통해 loss를 계산한 뒤, 그 결과를 `loss`에 저장합니다.

모델 검증 과정에서는 [model.eval()](https://pytorch.org/docs/stable/nn.html?highlight=eval#torch.nn.Module.eval)을 통해 모델을 evaluation 모드로 작동해줘야 함을 기억하시기 바랍니다. Batch normalization 과 Dropout은 훈련과 검증시에 작동하는 방식이 다르기 때문입니다. 평가가 끝난 후에는 다시 [model.train()](https://pytorch.org/docs/stable/nn.html?highlight=module%20train#torch.nn.Module.train)을 통해 train 모드로 바꿔줘야 하는 사실도 잊지 마세요.

**validation 함수를 작성해보세요! "<font color='45A07A'>## 코드 시작 ##</font>"과 "<font color='45A07A'>## 코드 종료 ##</font>" 사이의 <font color='075D37'>None</font> 부분을 채우시면 됩니다.**

In [ ]:
def validation(epoch, model, data_loader, criterion, device):
    print('Start validation #{}'.format(epoch))
    model.eval()
    with torch.no_grad():
        total_loss = 0
        cnt = 0
        for step, (sequence, target) in enumerate(data_loader):
            sequence = sequence.type(torch.float32)
            target = target.type(torch.float32)
            sequence, target = sequence.to(device), target.to(device)
            ## 코드 시작 ##
            outputs = None
            loss = None
            ## 코드 종료 ##
            total_loss += loss
            cnt += 1
        avrg_loss = total_loss / cnt
        print('Validation #{}  Average Loss: {:.4f}'.format(epoch, avrg_loss))
    model.train()
    return avrg_loss

### <font color='red'>[TODO] 코드 구현: 테스트 함수</font>

테스트 함수입니다. 다음을 읽고 코드를 완성해 보세요. 

- `validation` 함수와 마찬가지로 `model`에 입력 시퀀스(`sequence`)를 전달하고 출력 결과를 `outputs`에 저장하고, `criterion`을 통해 loss를 계산한 뒤, 그 결과를 `loss`에 저장합니다.

`test`함수를 이용하여 베이스라인과 모델의 성능비교를 진행합니다.

**test 함수 코드를 작성해보세요! "<font color='45A07A'>## 코드 시작 ##</font>"과 "<font color='45A07A'>## 코드 종료 ##</font>" 사이의 <font color='075D37'>None</font> 부분을 채우시면 됩니다.**

In [ ]:
def test(model, data_loader, criterion, baseline_loss, device):
    print('Start test..')
    model.eval()
    with torch.no_grad():
        total_loss = 0
        cnt = 0
        for step, (sequence, target) in enumerate(data_loader):
            sequence = sequence.type(torch.float32)
            target = target.type(torch.float32)
            sequence, target = sequence.to(device), target.to(device)
            ## 코드 시작 ##
            outputs = None
            loss = None
            ## 코드 종료 ##
            total_loss += loss
            cnt += 1
        avrg_loss = total_loss / cnt
        print('Test  Average Loss: {:.4f}  Baseline Loss: {:.4f}'.format(avrg_loss, baseline_loss))
        
    if avrg_loss < baseline_loss:
        print('베이스라인 성능을 뛰어 넘었습니다!')
    else:
        print('아쉽지만 베이스라인 성능을 넘지 못했습니다.')

## 9. 모델 저장 함수 정의

모델을 저장하는 함수입니다. 모델 저장은 [`torch.save`](https://pytorch.org/docs/stable/torch.html?highlight=save#torch.save) 함수를 통해 할 수 있습니다. 
[`nn.Module.state_dict`](https://pytorch.org/docs/stable/nn.html?highlight=state_dict#torch.nn.Module.state_dict)를 통해 Module, 즉 우리 모델의 파라미터를 가져올 수 있습니다. 이렇게 불러온 파라미터를 **check_point** 딕셔너리에 저장합니다. 그리고 이 **check_point**를 정해준 경로에 저장하면 됩니다. 

[PyTorch 공식 튜토리얼](https://pytorch.org/tutorials/beginner/saving_loading_models.html)에서 모델을 저장하고 불러오는 방법에 대한 예제를 확인하실 수 있습니다. 

torch.save는 단순히 모델의 파라미터만 저장하는 함수가 아닙니다. 어떤 파이썬 객체든 저장할 수 있습니다. 그래서 경우에 따라 **check_point** 딕셔너리에 모델의 파라미터 뿐만 아니라 다른 여러 가지 필요한 정보를 저장할 수도 있습니다. 예를 들어 총 몇 에폭동안 학습한 모델인지 그 정보도 저장할 수 있겠죠? 

### <font color='red'>[TODO] 코드 구현</font>

다음을 읽고 코드를 완성해보세요.
- torch.save를 통해 `output_path` 경로에 `check_point` 를 저장하세요.

**save_model 함수를 작성해보세요! "<font color='45A07A'>## 코드 시작 ##</font>"과 "<font color='45A07A'>## 코드 종료 ##</font>" 사이의 <font color='075D37'>None</font> 부분을 채우시면 됩니다.**

In [ ]:
def save_model(model, saved_dir, file_name='best_model.pt'):
    os.makedirs(saved_dir, exist_ok=True)
    check_point = {
        'net': model.state_dict()
    }
    output_path = os.path.join(saved_dir, file_name)
    ## 코드 시작 ##
    None
    ## 코드 종료 ##

## 10. 모델 생성 및 Loss function, Optimizer 정의

생성한 모델을 학습 시키기 위해서 손실함수를 정의해야 합니다. 뉴럴네트워크는 경사하강(gradient descent)방법을 이용하여 손실함수의 값을 줄이는 방향으로 파라미터를 갱신(update) 하게 됩니다. 또한 효과적인 경사하강 방법을 적용하기 위해 옵티마이져를 함께 사용할 겁니다.

### <font color='red'>[TODO] 코드 구현</font>

다음을 읽고 코드를 완성해보세요. 

1. **<7. 네트워크 설계>** 에서 정의한 SimpleLSTM class를 통해 모델을 생성하고 이를 `model` 변수에 저장합니다.
2. 연속된 값을 예측하는 회기(regression) 문제에서는 보통 Mean Squared Error(MSE) loss function을 사용합니다. [nn.MSELoss](https://pytorch.org/docs/stable/nn.html?highlight=mseloss#torch.nn.MSELoss)를 통해 MSE loss function을 선언하고 이를 **criterion** 변수에 저장합니다.
3. 이번 실습에서는 Adam 옵티마이저를 통해 파라미터를 업데이트 하겠습니다. Adam optimizer([`torch.optim.Adam`](https://pytorch.org/docs/stable/optim.html?highlight=adam#torch.optim.Adam))를 `optimizer` 변수에 저장합니다. **<2. 하이퍼파라미터 세팅>** 에서 정의한 `learning_rate` 를 사용하세요.

`val_every`는 검증을 몇 에폭마다 진행할지 정하는 변수입니다. `saved_dir`은 모델이 저장될 디렉토리의 경로입니다. 

**모델을 생성하고 손실함수 및 옵티마이저를 작성해보세요! "<font color='45A07A'>## 코드 시작 ##</font>"과 "<font color='45A07A'>## 코드 종료 ##</font>" 사이의 <font color='075D37'>None</font> 부분을 채우시면 됩니다.**

In [ ]:
torch.manual_seed(7777) # 일관된 weight initialization을 위한 random seed 설정
## 코드 시작 ##
model = None          # 위의 설명 1. 을 참고하여 None을 채우세요.
model = model.to(device)
criterion = None      # 위의 설명 2. 를 참고하여 None을 채우세요.
optimizer = None      # 위의 설명 3. 을 참고하여 None을 채우세요.
## 코드 종료 ##
val_every = 1
saved_dir = './saved/LSTM'

아래의 코드를 실행해 코드를 성공적으로 완성했는지 확인해보세요. 

별다른 문제가 없다면 이어서 진행하면 됩니다.

In [ ]:
checker.loss_func_check(criterion)
checker.optim_check(optimizer)

## 11. Training

**<8. train, validation, test 함수 정의>** 에서 작성한 `train` 함수를 통해 모델 학습을 진행합니다. LSTM과 같은 recurrent layer는 타임스텝을 따라 계산이 순차적으로 진행되기 때문에 상대적으로 학습 시간이 많이 소요됩니다. 따라서 시간 여유가 없는 분들은 모델 학습이 적당히 진행된다는 정도만 확인하고 다음 단계로 넘어가셔도 됩니다.

만약 한 에폭 이상이 지나도록 loss가 전혀 감소하는 기미가 보이지 않는다면 이전의 구현에 문제가 있을 가능성이 높으니 코드를 다시 검토해주시기 바랍니다.

또한, 모델 저장 코드를 제대로 구현했다면 첫 에폭 학습후에 ./saved/LSTM 경로에 best_model.pt 파일이 저장되어 있어야 합니다. 만약에 파일이 존재하지 않는다면 모델 저장 코드를 다시 확인하시기 바랍니다.

In [ ]:
train(num_epochs, model, train_loader, criterion, optimizer, saved_dir, val_every, device)

## 12. 저장된 모델 불러오기 및 테스트

학습한 모델의 성능을 테스트합니다. 저장한 모델 파일을 [`torch.load`](https://pytorch.org/docs/stable/torch.html?highlight=load#torch.load)를 통해 불러옵니다. 위에서 학습을 끝까지 진행하지 않았다면, 아래의 주석 처리된 부분을 주석 해제하면, 제공해드린 미리 학습시킨 모델을 불러올 수 있습니다. 

이렇게 불러오면 우리가 얻게 되는 건 아까 저장한 **check_point** 딕셔너리입니다. 딕셔너리에 저장한 모델의 파라미터는 **'net'** key에 저장해두었습니다. 이를 불러와 **state_dict**에 저장합니다. 이렇게 불러온 모델의 파라미터를 모델에 실제로 로드하기 위해서는 [`nn.Module.load_state_dict`](https://pytorch.org/docs/stable/torch.html?highlight=load#torch.load)를 사용하면 됩니다. 

### <font color='red'>[TODO] 코드 구현</font>
다음을 읽고 코드를 완성해보세요.

1. `model_path`의 경로에 있는 모델 파일을 로드하여, 이를 `check_point` 변수에 저장합니다. 또한, 미리저장된 모델이 GPU로 학습했는데 CPU 로 불러올 경우, 파이토치에서 모델을 불러오는 함수에 `map_location` 인자에 `device` 정보를 전달해야 적용됩니다. (즉, 함수에 `map_location=device` 가 되어야 합니다.) `device` 변수는 **<1. Package load>** 에서 이미 선언했습니다.
2. `check_point` 딕셔너리에 접근하여 모델의 파라미터를 `state_dict` 변수에 저장합니다. 접근을 위한 딕셔너리의 키값은 'net' 입니다.
3. `state_dict`의 파라미터들을 새로 선언한 모델(`model`)에 로드합니다.

**새로 선언한 `model` 에 저장된 파라미터를 불러오는 코드를 작성해보세요! "<font color='45A07A'>## 코드 시작 ##</font>"과 "<font color='45A07A'>## 코드 종료 ##</font>" 사이의 <font color='075D37'>None</font> 부분을 채우시면 됩니다.**

In [ ]:
model_path = './saved/LSTM/best_model.pt'
# model_path = './saved/pretrained/LSTM/best_model.pt' # 모델 학습을 끝까지 진행하지 않은 경우에 사용
model = SimpleLSTM().to(device) # 아래의 모델 불러오기를 정확히 구현했는지 확인하기 위해 새로 모델을 선언하여 학습 이전 상태로 초기화

## 코드 시작 ##
checkpoint = None    # 위의 설명 1. 을 참고하여 None을 채우세요.
state_dict = None    # 위의 설명 2. 를 참고하여 None을 채우세요.
None                 # 위의 설명 3. 을 참고하여 None을 채우세요.
## 코드 종료 ##

마지막으로 모델의 성능을 테스트합니다. 베이스라인 성능을 뛰어 넘었다는 문구가 나오면 성공적으로 진행한 것입니다. 

In [ ]:
test(model, test_loader, criterion, baseline_loss, device)

학습된 모델의 예측 기온과 실제 기온을 몇가지 살펴보면 다음과 같습니다. 

In [ ]:
for i in range(15):
    data_idx = np.random.randint(len(test_data))
    sequence = test_data[data_idx][0]
    sequence = torch.Tensor(sequence).unsqueeze(0).to(device)
    
    pred = model(sequence)
    pred = pred.item() * test_data.std[0] + test_data.mean[0]
    target = test_data[data_idx][1][0] * test_data.std[0] + test_data.mean[0]
    print('예측 기온: {:.1f} / 실제 기온: {:.1f}'.format(pred, target))

어떠신가요? 잘 맞는것 같나요? 모델 구조의 은닉층 크기(`hidden_size`) 등 다양한 하이퍼파라미터를 조절해서 더 좋은 성능을 내는 모델을 만들어보세요!

## 13. Summary

이로써 마지막 RNN 프로젝트를 완료했습니다. 고생하셨습니다! 

우리는 이번 실습을 통해 다음과 같은 내용을 학습했습니다.

- 날씨와 같은 시계열 정보를 다룰 수 있다.
- RNN을 설계하고 시간 순서상 미래의 정보를 예측하는 모델을 학습시킬 수 있다.
- 상식적인 수준의 베이스라인을 도입하여 학습한 모델의 성능을 검증할 수 있다.

---

# Self-Review

학습 환경에 맞춰 알맞는 제출방법을 실행하세요!

### 로컬 환경 실행자

1. 모든 실습 완료 후, Jupyter Notebook 을 `Ctrl+S` 혹은 `File > Save and checkpoint`로 저장합니다.
2. 제일 하단의 코드를 실행합니다. 주의할 점은 Jupyter Notebook 의 파일이름을 수정하시면 안됩니다! 만약에 노트북 이름을 수정했다면 "pytorch-rnn-project" 로 바꿔주시길 바랍니다. 모든 평가 기준을 통과하면, 함수 실행 후 프로젝트 "submit" 디렉토리와 압축된 "submit.zip"이 생깁니다. "cnn_submission.tsv" 파일을 열고 모두 Pass 했는지 확인해보세요!
    * "rnn_submission.tsv" : 평가 기준표에 근거해 각 세부항목의 통과여부(Pass/Fail) 파일
    * "rnn_submission.html" : 여러분이 작성한 Jupyter Notebook 을 html 형식으로 전환한 파일
3. 코드 실행결과 안내에 따라서 `submit.zip` 파일을 확인하시고 제출해주시길 바랍니다.

### Colab 환경 실행자

1. 모든 실습 완료 후, Jupyter Notebook 을 `Ctrl+S` 로 저장합니다.
2. 제일 하단의 코드를 실행합니다. 코드 실행결과 안내에 따라서 재작성하거나 다음스텝으로 넘어갑니다. 모든 평가 기준을 통과하면, 함수 실행 후 프로젝트 "submit" 디렉토리와 압축된 "rnn_submission.tsv"만 생깁니다. "rnn_submission.tsv" 파일을 열고 모두 Pass 했는지 확인해보세요!
    * "rnn_submission.tsv" : 평가 기준표에 근거해 각 세부항목의 통과여부(Pass/Fail) 파일
3. 프로젝트를 저장한 드라이브의 `submit` 폴더에서 `rnn_submission.tsv` 파일을 다운 받습니다.
4. Colab Notebook 에서 `파일 > .ipynb 다운로드`를 통해서 노트북을 다운로드 받습니다.
5. 로컬에서 Jupyter Notebook 프로그램을 실행시킵니다. 
6. 4번 스텝에서 다운받은 노트북을 열고 `File > Download as > HTML(.html)` 로 재 다운로드 합니다.
7. 3번 스텝에서 받은 파일과 6번 스텝에서 받은 파일을 하나의 폴더에 넣고, `submit.zip` 이라는 이름으로 압축하고 제출해주시길 바랍니다.

In [ ]:
import check_util.submit as submit
submit.process_submit()